In [1]:
# !echo swamiji123 | sudo -S apt-get -y install python3.10-dev

In [2]:
#@title Unzip File From Drive

!unzip -n /workspace/sph-voice-clone/drive/MyDrive/assets/KiwKiw.zip -d /workspace/sph-voice-clone/
%cd /workspace/sph-voice-clone/KiwKiw
%pip install pickleshare
%pip install -r requirements.txt 


Archive:  /workspace/sph-voice-clone/drive/MyDrive/assets/KiwKiw.zip
/workspace/sph-voice-clone/KiwKiw
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Ignoring onnxruntime: markers 'sys_platform == "darwin"' don't match your environment
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version

In [3]:
import os

# @title Create Model Folder
model_name = "sphVoice" # @param {type:"string"}
exp_dir = model_name
dataset = "/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice" # @param {type:"string"}
sample_rate = "48000" # @param ["40000", "48000"]
ksample_rate = "48k"
if sample_rate == "40000":
  ksample_rate = "40k"
else:
  ksample_rate = "48k"
version = "v2" # @param ["v1", "v2"]
version19 = version

f0method = "rmvpe_gpu" # @param ["pm", "dio", "harvest", "rmvpe", "rmvpe_gpu"]

save_frequency = 50 # @param {type:"slider", min:0, max:50, step:1}
epoch = 250 # @param {type:"integer"}
batch_size = "40" # @param {type:"string"}
cache_gpu = True # @param {type:"boolean"}

now_dir = "/workspace/sph-voice-clone/KiwKiw"

os.makedirs("%s/logs/%s" % (now_dir, exp_dir), exist_ok=True)
f = open("%s/logs/%s/preprocess.log" % (now_dir, exp_dir), "w")
os.makedirs("%s/logs/%s" % (now_dir, exp_dir), exist_ok=True)
f = open("%s/logs/%s/extract_f0_feature.log" % (now_dir, exp_dir), "w")
f.close()

In [4]:
# @title Process Data
command = f"python infer/modules/train/preprocess.py '{dataset}' {sample_rate} 2 '{now_dir}/logs/{exp_dir}' False 3.0"
print(command)
!{command}

python infer/modules/train/preprocess.py '/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice' 48000 2 '/workspace/sph-voice-clone/KiwKiw/logs/sphVoice' False 3.0
/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice 48000 2 /workspace/sph-voice-clone/KiwKiw/logs/sphVoice False 3.0
start preprocess
/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice/split_0.wav	-> Success
/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice/split_1.wav	-> Success
/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice/split_10.wav	-> Success
/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice/split_100.wav	-> Success
/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice/split_1000.wav	-> Success
/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice/split_1001.wav	-> Success
/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice/split_1002.wav	-> Success
/workspace/sph-voice-clone/drive/MyDrive/dataset/sphVoice/split_1003.wav	-> Success
/workspace/sph-voice-clone/

In [5]:
# @title Feature Extraction

if f0method != "rmvpe_gpu":
  command = f"python infer/modules/train/extract/extract_f0_print.py '{now_dir}/logs/{exp_dir}' 2 '{f0method}'"
else:
  command = f"python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 '{now_dir}/logs/{exp_dir}' True"
print(command)
!{command}

command = f"python infer/modules/train/extract_feature_print.py cuda:0 1 0 0 '{now_dir}/logs/{exp_dir}' '{version}' False"
print(command)
!{command}

python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 '/workspace/sph-voice-clone/KiwKiw/logs/sphVoice' True
infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 /workspace/sph-voice-clone/KiwKiw/logs/sphVoice True
todo-f0-2737
f0ing,now-0,all-2737,-/workspace/sph-voice-clone/KiwKiw/logs/sphVoice/1_16k_wavs/0_1.wav
f0ing,now-547,all-2737,-/workspace/sph-voice-clone/KiwKiw/logs/sphVoice/1_16k_wavs/208_2.wav
f0ing,now-1094,all-2737,-/workspace/sph-voice-clone/KiwKiw/logs/sphVoice/1_16k_wavs/405_1.wav
f0ing,now-1641,all-2737,-/workspace/sph-voice-clone/KiwKiw/logs/sphVoice/1_16k_wavs/595_2.wav
f0ing,now-2188,all-2737,-/workspace/sph-voice-clone/KiwKiw/logs/sphVoice/1_16k_wavs/78_2.wav
f0ing,now-2735,all-2737,-/workspace/sph-voice-clone/KiwKiw/logs/sphVoice/1_16k_wavs/9_1.wav
python infer/modules/train/extract_feature_print.py cuda:0 1 0 0 '/workspace/sph-voice-clone/KiwKiw/logs/sphVoice' 'v2' False
infer/modules/train/extract_feature_print.py cuda:0 1 0 0 /workspace/sph-voice-clon

In [6]:
import os
import numpy as np
import faiss
from sklearn.cluster import MiniBatchKMeans
import traceback
import logging

# Set up logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "请先进行特征提取!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "请先进行特征提取！"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("training")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("adding")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "成功构建索引，added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

result_generator = train_index("example_exp_dir", "v1")

for result in result_generator:
    print(result)

In [ ]:
# @title Train Model

from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT

# Remove the logging setup

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    # Replace logger.debug, logger.info with print statements
    print("Write filelist done")
    print("Use gpus:", str(gpus16))
    if pretrained_G14 == "":
        print("No pretrained Generator")
    if pretrained_D15 == "":
        print("No pretrained Discriminator")
    if version19 == "v1" or sr2 == "40k":
        config_path = "configs/v1/%s.json" % sr2
    else:
        config_path = "configs/v2/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    # Use PIPE to capture the output and error streams
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    # Print the command's output as it runs
    for line in p.stdout:
        print(line.strip())

    # Wait for the process to finish
    p.wait()
    return "训练结束, 您可查看控制台训练日志或实验文件夹下的train.log"

if version == 'v1':
    if ksample_rate == '40k':
        G_path = 'assets/pretrained/f0G40k.pth'
        D_path = 'assets/pretrained/f0D40k.pth'
    elif ksample_rate == '48k':
        G_path = 'assets/pretrained/f0G48k.pth'
        D_path = 'assets/pretrained/f0D48k.pth'
elif version == 'v2':
    if ksample_rate == '40k':
        G_path = 'assets/pretrained_v2/f0G40k.pth'
        D_path = 'assets/pretrained_v2/f0D40k.pth'
    elif ksample_rate == '48k':
        G_path = 'assets/pretrained_v2/f0G48k.pth'
        D_path = 'assets/pretrained_v2/f0D48k.pth'

result_generator = click_train(
    exp_dir,
    ksample_rate,
    True,
    0,
    save_frequency,
    epoch,
    batch_size,
    True,
    G_path,
    D_path,
    0,
    cache_gpu,
    False,
    version,
)
print(result_generator)



Write filelist done
Use gpus: 0
INFO:sphVoice:{'data': {'filter_length': 2048, 'hop_length': 480, 'max_wav_value': 32768.0, 'mel_fmax': None, 'mel_fmin': 0.0, 'n_mel_channels': 128, 'sampling_rate': 48000, 'win_length': 2048, 'training_files': './logs/sphVoice/filelist.txt'}, 'model': {'filter_channels': 768, 'gin_channels': 256, 'hidden_channels': 192, 'inter_channels': 192, 'kernel_size': 3, 'n_heads': 2, 'n_layers': 6, 'p_dropout': 0, 'resblock': '1', 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'resblock_kernel_sizes': [3, 7, 11], 'spk_embed_dim': 109, 'upsample_initial_channel': 512, 'upsample_kernel_sizes': [24, 20, 4, 4], 'upsample_rates': [12, 10, 2, 2], 'use_spectral_norm': False}, 'train': {'batch_size': 40, 'betas': [0.8, 0.99], 'c_kl': 1.0, 'c_mel': 45, 'epochs': 20000, 'eps': 1e-09, 'fp16_run': True, 'init_lr_ratio': 1, 'learning_rate': 0.0001, 'log_interval': 200, 'lr_decay': 0.999875, 'seed': 1234, 'segment_size': 17280, 'warmup_epochs': 0}, 'model_dir':

In [ ]:
#@title Copy Training Result to Drive
!mkdir -p /workspace/sph-voice-clone/drive/MyDrive/Model/{exp_dir}

!cp /workspace/sph-voice-clone/KiwKiw/logs/{exp_dir}/added_*.index /workspace/sph-voice-clone/drive/MyDrive/Model/{exp_dir}
!cp /workspace/sph-voice-clone/KiwKiw/logs/{exp_dir}/total_*.npy /workspace/sph-voice-clone/drive/MyDrive/Model/{exp_dir}
!cp /workspace/sph-voice-clone/KiwKiw/assets/weights/{exp_dir}.pth /workspace/sph-voice-clone/drive/MyDrive/Model/{exp_dir}

%cd /workspace/sph-voice-clone/drive/MyDrive/Model/{exp_dir}
!zip {exp_dir}.zip added_*.index total_*.npy {exp_dir}.pth
%cd /workspace/sph-voice-clone/KiwKiw